In [1]:
# /Notebooks/001-Tutorial-Connecting-to-Data-Source.ipynb

# Tutorial: Connecting and Testing Alpaca Connections

#  Setup & Imports

In [2]:
# Standard libraries
from datetime import datetime, timedelta
import time
import sys
from pathlib import Path
import os

# Non-standard libraries
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

# Import paths from the config module (now in src/)
from src.config import BASE_DIR as CONFIG_BASE_DIR

# Project-specific imports
from src.utils import get_alpaca_client, fetch_alpaca_stock_tickers, fetch_alpaca_historical_data, fetch_alpaca_latest_bars
from credentials import ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPAKA_ENDPOINT_URL

# Initialize Alpaca client
alpaca = get_alpaca_client()

Project root added to sys.path: /workspaces/stat_656_autotrader


# Connect and Fetch Alpaca Tickers

In [3]:
# Fetch Alpaca tickers using existing utility
stock_tickers = fetch_alpaca_stock_tickers(alpaca)

print(f"Total tickers fetched: {len(stock_tickers)}")
print("First 20 tickers:", stock_tickers[:20])

Error fetching tickers: request is not authorized
Total tickers fetched: 0
First 20 tickers: []


# Configure Test Parameters

In [4]:
years_back = 5
subset_size = 175
subset_tickers = stock_tickers[:subset_size]

start_date = "2002-01-01"
end_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
trading_days_back = years_back * 252  # Approximate number of trading days

print(f"Fetching OHLC data from {start_date} to {end_date} (approx. {trading_days_back} trading days)")

Fetching OHLC data from 2002-01-01 to 2025-04-06 (approx. 1260 trading days)


# Fetch Historical Data (OHLC)

In [5]:
# Use existing fetch utility for historical OHLC
historical_df = fetch_alpaca_historical_data(alpaca, subset_tickers, start_date, end_date, years_back)

print(f"\nFetched {len(historical_df)} rows of historical OHLC data.")
print(historical_df.head(10))

Alpaca Download Progress: 0it [00:00, ?it/s]


ZeroDivisionError: float division by zero

# Fetch Yesterday's OHLC

In [ ]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

yesterday_df = fetch_alpaca_historical_data(alpaca, subset_tickers, yesterday, yesterday)

print(f"\nFetched OHLC data for yesterday ({yesterday}):")
print(yesterday_df.head(10))

# Fetch Today's Open Prices

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")
market_open_dt = pd.Timestamp(f"{today}T09:30:00-05:00")

open_prices_df = pd.DataFrame()

for ticker in tqdm(subset_tickers, desc="Fetching Today's Open Prices"):
    bars = alpaca.get_bars(ticker, "1Min", market_open_dt.isoformat(), (market_open_dt + timedelta(minutes=1)).isoformat()).df
    if not bars.empty:
        bars.reset_index(inplace=True)
        open_prices_df = pd.concat([
            open_prices_df,
            pd.DataFrame({
                'ticker': [ticker],
                'date': [today],
                'open': [bars.iloc[0]['open']]
            })
        ])
    time.sleep(0.3)  # Slightly faster, but avoid hitting rate limits

print(f"\nToday's Open Prices ({today}):")
print(open_prices_df.head(10))

# Batch Fetch Real-time latest Bars

In [ ]:
real_time_df = fetch_alpaca_latest_bars(alpaca, subset_tickers)

print("\nReal-Time Latest Bars (Market Open):")
print(real_time_df.head(10))

# Performance Summary

In [ ]:
# Simple diagnostics
def performance_summary(start_time, num_tickers):
    total_seconds = time.time() - start_time
    seconds_per_ticker = total_seconds / num_tickers
    estimated_time_all = (seconds_per_ticker * len(stock_tickers)) / 60  # in minutes

    print(f"\nProcessed {num_tickers} tickers in {total_seconds:.2f} seconds.")
    print(f"Average time per ticker: {seconds_per_ticker:.2f} sec.")
    print(f"Estimated time for all tickers: {estimated_time_all:.2f} mins.")

# Example Usage:
start_time = time.time()
_ = fetch_alpaca_latest_bars(alpaca, subset_tickers[:10])
performance_summary(start_time, 10)